In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import key_dict

#gmaps key 
g_key = key_dict['g_key']

 Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../WeatherPy/output/weather_info_df.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,80.13,71,0,7.54,PF,1616357944
1,Ushuaia,-54.8000,-68.3000,55.40,58,40,18.41,AR,1616357707
2,Busselton,-33.6500,115.3333,64.99,95,80,3.00,AU,1616357988
3,Sakakah,29.9697,40.2064,73.40,17,0,4.61,SA,1616358327
4,Saldanha,-33.0117,17.9442,62.60,67,0,6.91,ZA,1616358034


 Humidity Heatmap

Configure gmaps.Use the Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store lat, lng into locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

#humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

 Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.Drop any rows will null values.

In [6]:
#create ideal weather df
ideal_weather = weather_df

# A max temperature < 80 degrees > 70.
ideal_weather =ideal_weather.loc[(ideal_weather["Max Temp"] < 80) & (ideal_weather["Max Temp"] > 70)]

# Wind speed < 10 mph.
ideal_weather = ideal_weather.loc[ideal_weather["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather = ideal_weather.loc[ideal_weather["Clouds"] == 0]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

ideal_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
3,Sakakah,29.9697,40.2064,73.40,17,0,4.61,SA,1616358327
130,Akaltara,22.0167,82.4333,74.57,30,0,2.19,IN,1616358362
149,Bandarbeyla,9.4942,50.8122,77.79,69,0,7.83,SO,1616358068
163,Kālāgarh Project Colony,29.5000,78.7500,70.66,26,0,5.12,IN,1616358372
205,Mercedes,-34.6515,-59.4307,75.99,61,0,1.01,AR,1616358385


 Hotel Map

Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

In [7]:
#create hotel df
hotel_df = pd.DataFrame(weather_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Rikitea,PF,-23.1203,-134.9692,
1,Ushuaia,AR,-54.8000,-68.3000,
2,Busselton,AU,-33.6500,115.3333,
3,Sakakah,SA,29.9697,40.2064,
4,Saldanha,ZA,-33.0117,17.9442,


In [8]:
# parameters
parameters = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]


    parameters["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(url, params=parameters).json()
    
    #get results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Rikitea.
Closest hotel is Pension Maro'i.
------------
Retrieving Results for Index 1: Ushuaia.
Closest hotel is Albatros Hotel.
------------
Retrieving Results for Index 2: Busselton.
Closest hotel is Observatory Guest House.
------------
Retrieving Results for Index 3: Sakakah.
Closest hotel is Fakhamat Al Orjoana Apartment.
------------
Retrieving Results for Index 4: Saldanha.
Closest hotel is Blue Bay Lodge.
------------
Retrieving Results for Index 5: Kayes.
Missing field/result... skipping.
------------
Retrieving Results for Index 6: Avarua.
Closest hotel is Paradise Inn.
------------
Retrieving Results for Index 7: Punta Arenas.
Closest hotel is Hotel Dreams Del Estrecho.
------------
Retrieving Results for Index 8: Shitkino.
Missing field/result... skipping.
------------
Retrieving Results for Index 9: Arman'.
Missing field/result... skipping.
------------
Retrieving Results for Index 10: Miandrivazo.
Closest hotel is Hôtel Arc En Ciel Miandriv

Closest hotel is Gostinitsa.
------------
Retrieving Results for Index 86: Sayyan.
Closest hotel is متنزه و بحيرة سيان.
------------
Retrieving Results for Index 87: Whitehorse.
Closest hotel is The Sternwheeler Hotel & Conference Centre.
------------
Retrieving Results for Index 88: New Norfolk.
Closest hotel is The Shingles Riverside Cottages.
------------
Retrieving Results for Index 89: Cockburn Town.
Closest hotel is Osprey Beach Hotel.
------------
Retrieving Results for Index 90: Bǝrdǝ.
Closest hotel is Prins Motel Restoran.
------------
Retrieving Results for Index 91: Vardø.
Closest hotel is Vardø Hotel AS.
------------
Retrieving Results for Index 92: Fortuna.
Closest hotel is Super 8 by Wyndham Fortuna.
------------
Retrieving Results for Index 93: Esperance.
Closest hotel is Hospitality Esperance.
------------
Retrieving Results for Index 94: Bryan.
Closest hotel is Best Western Premier Bryan College Station.
------------
Retrieving Results for Index 95: Port Augusta.
Close

Closest hotel is ProDAF/Diffa.
------------
Retrieving Results for Index 171: Tiksi.
Closest hotel is Arktika.
------------
Retrieving Results for Index 172: Ilebo.
Closest hotel is Hôtel BUMAK.
------------
Retrieving Results for Index 173: Coahuayana Viejo.
Closest hotel is El rranchito michuacan de ocampos.
------------
Retrieving Results for Index 174: Ubari.
Closest hotel is فندق أوباري.
------------
Retrieving Results for Index 175: Jiangyou.
Closest hotel is Shangping Shuyuan Inn.
------------
Retrieving Results for Index 176: El Alto.
Closest hotel is Black Marlin Restaurant & Hospedaje.
------------
Retrieving Results for Index 177: Tam Ky.
Closest hotel is Khách Sạn Đông Á.
------------
Retrieving Results for Index 178: Faanui.
Closest hotel is Bora-Bora Pearl Beach Resort & Spa.
------------
Retrieving Results for Index 179: Klaksvík.
Closest hotel is Airbnb rooms.
------------
Retrieving Results for Index 180: Port Elizabeth.
Closest hotel is 39 On Nile Guest House.
-------

Closest hotel is Hotel palance.
------------
Retrieving Results for Index 258: Manitouwadge.
Closest hotel is Select Inn.
------------
Retrieving Results for Index 259: Barrow.
Closest hotel is King Eider Inn.
------------
Retrieving Results for Index 260: Inhambane.
Closest hotel is Oceano.
------------
Retrieving Results for Index 261: Kolondiéba.
Closest hotel is Hotel Dakan.
------------
Retrieving Results for Index 262: Takoradze.
Closest hotel is Raybow International Hotel.
------------
Retrieving Results for Index 263: Sept-Îles.
Closest hotel is Comfort Inn.
------------
Retrieving Results for Index 264: Jardim.
Closest hotel is Estancia Hotel - Jardim MS.
------------
Retrieving Results for Index 265: Tai’an.
Closest hotel is Taishan International Youth Hostel.
------------
Retrieving Results for Index 266: Berlevåg.
Closest hotel is Kvitbrakka.
------------
Retrieving Results for Index 267: Dongargaon.
Closest hotel is S r farmhouse.
------------
Retrieving Results for Index 

Closest hotel is Yeet Hotel.
------------
Retrieving Results for Index 344: Kapoeta.
Closest hotel is Eastgate Hotel.
------------
Retrieving Results for Index 345: Coquimbo.
Closest hotel is Punto de encuentro.
------------
Retrieving Results for Index 346: Ternate.
Closest hotel is Grand Dafam Bela Ternate.
------------
Retrieving Results for Index 347: Nelson Bay.
Closest hotel is Mantra Nelson Bay.
------------
Retrieving Results for Index 348: Atambua.
Closest hotel is Pastori 1 Jemaat Polycarpus.
------------
Retrieving Results for Index 349: Kologriv.
Closest hotel is Postoyalyy Dvor.
------------
Retrieving Results for Index 350: Yerbogachën.
Missing field/result... skipping.
------------
Retrieving Results for Index 351: Plouzané.
Closest hotel is Lymina.
------------
Retrieving Results for Index 352: Cayenne.
Closest hotel is Hôtel Le Dronmi.
------------
Retrieving Results for Index 353: Verkhnyaya Inta.
Missing field/result... skipping.
------------
Retrieving Results for I

Closest hotel is Radisson Blu Polar Hotel, Spitsbergen.
------------
Retrieving Results for Index 430: Nishihara.
Closest hotel is Minshuku Agaihama.
------------
Retrieving Results for Index 431: ‘Ewa Beach.
Closest hotel is Uncle Junior Robert.
------------
Retrieving Results for Index 432: Sinnamary.
Closest hotel is ALETHEIA Appartement Sinnamary.
------------
Retrieving Results for Index 433: Forsytheganj.
Closest hotel is Hotel Svetlogorsk JBIK.
------------
Retrieving Results for Index 434: Prince George.
Closest hotel is Canadas Best Value Inn Prince George.
------------
Retrieving Results for Index 435: Mitsamiouli.
Closest hotel is Foyer ADM.
------------
Retrieving Results for Index 436: Tuy Hoa.
Closest hotel is CenDeluxe Hotel Tuy Hoa.
------------
Retrieving Results for Index 437: Kulachi.
Closest hotel is Technical Zafran Mobile Zon kulachi.
------------
Retrieving Results for Index 438: Deputatsky.
Closest hotel is Baza Otdykha.
------------
Retrieving Results for Index

Closest hotel is Taloa Heights.
------------
Retrieving Results for Index 514: Alexandria.
Closest hotel is Royal Crown Hotel.
------------
Retrieving Results for Index 515: Boguchany.
Closest hotel is Uyut.
------------
Retrieving Results for Index 516: Columbus.
Closest hotel is DoubleTree Suites by Hilton Hotel Columbus Downtown.
------------
Retrieving Results for Index 517: Paita.
Closest hotel is Hotel Náutico de Paita.
------------
Retrieving Results for Index 518: Indramayu.
Closest hotel is Grand Trisula Hotel.
------------
Retrieving Results for Index 519: Wolfenbüttel.
Closest hotel is Parkhotel Altes Kaffeehaus GmbH.
------------
Retrieving Results for Index 520: Aginskoye.
Closest hotel is Кочевник.
------------
Retrieving Results for Index 521: Bolaños de Calatrava.
Closest hotel is Hotel Almagro.
------------
Retrieving Results for Index 522: Sur.
Closest hotel is Sur Plaza Hotel.
------------
Retrieving Results for Index 523: Jalingo.
Closest hotel is Debright Motel.
--

In [ ]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]